# 1. Basic Notation

## 1.1 Graph Representation

- Let the set of nodes (junctions) be $\mathcal{N}$, with n=$|\mathcal{N}|$
- Let the set of branches (pipes, pumps, valves etc.) be $\mathcal{E}$, with $e=|\mathcal{E}|$

Each branch $e\in\mathcal{E}$ connects two nodes. We define a direction (orientation) for each branch. This orientation can be chosen arbitrarily as long as it is kept consistent throughout the equations (it does not have to match actual flow direction in practice; it’s just a bookkeeping device).

## 1.2 Incidence Matrix

Define the incidence matrix $A\in\mathbb{R}^{n\times e}$ by:

$$
A_{i,j} = 
\begin{cases}
+1 & \text{if branch }j\text{ enters node } i, \\
-1 & \text{if branch }j\text{ leaves node } i, \\
0  & \text{otherwise.}
\end{cases}
$$

## 1.3 Variables

- **Nodal pressures**: Let $p_i$ denote the pressure at node $i$. Collect these as a vector:

$$
\vec{p} =
\begin{bmatrix}
p_1 \\
p_2 \\
\vdots \\
p_n
\end{bmatrix}
$$

Often, one node is chosen as a reference (ground) with $p_\text{ref}=0$, reducing the number of unknown pressures. E.g., a node exposed to atmosphere.

- **Branch flowrates**: Let $q_j$ denote the volumetric flowrate (or mass flowrate as per the decided convention) in branch $j$. Collect these as a vector:
$$
\vec{q} =
\begin{bmatrix}
q_1 \\
q_2 \\
\vdots \\
q_e
\end{bmatrix}
$$

A positive $q_j$ means a positive flow in the direction of the branch $j$; negative $q_j$ means a flow in the opposite direction.

- **Branch pressure change**: Let $\Delta p_j$ denote the pressure change in the direction of the branch $j$. By definition,

$$\Delta p_j = p_\text{node at "outlet" of j} - p_\text{node at "inlet" of j}$$

Collect these as a vector:

$$
\vec{\Delta p} =
\begin{bmatrix}
\Delta p_1 \\
\Delta p_2\\
\vdots \\
\Delta p_e
\end{bmatrix}
$$

Note that there is a standard relationship between the nodal pressure vector, $\vec{p}$ and $\vec{\Delta p}$ via the mapping incidence matrix $A$. We can then relate these via:

$$\vec{\Delta p}=A^\top\vec{p}$$

if the convention of $A_{i,j}=+1$ when branch $j$ ends at node $i$ and -1 when is starts at node $i$ (as above).


# 2. Steady-State Formulation

## 2.1 Mass (or Volume) Balance at Each Node (KCL)

In steady state, there is no accumulation at each node, so the net flow into each node is zero (assuming constant fluid density if using volume for branch flow):

$$A\vec{q}=0$$

This is a direct analog of Kirchoff's Current Law (KCL) in circuit theory.

## 2.2 Pressure-Flow Relationships in Each Branch

For each branch $j$, there is a contituitive relation between flow, $q_j$, and pressure drop $\Delta p_j$. For example:

- **(Linear) Hydraulic Resistance** (e.g., laminar flow major losses in a pipe): Analogous to ohms law in circuits, becomes more complex (and nonlinear) for turbulent losses, controlled valve pressure drops/ cv curves etc.

$$\Delta p_j = R_j q_j$$

- **Pump or "voltage source"**: May either be a fixed $\Delta p_j = \text{constant}$ or a more complex pump curve $\Delta p_j = f(q_j)$.

In general we have a nonlinear relationship of the form:

$$\vec{\Delta p} = \mathbf{f}(\vec{q})$$

For a purely linear system, we simply have:

$\vec{\Delta p} = R\vec{q}$ where $R$ is a matrix of resistances.

## 2.3 Assembling the Steady-State System

We then have a system of equations:

1. $\vec{\Delta p} = A^\top \vec{p}$ (pressure differences from nodal pressures)
2. $\vec{\Delta p} = f(\vec{q})$ (branch constitutive laws)
3. $A \vec{q} = 0$ (mass continuity)

Generalised in matrix form:

$$
\begin{bmatrix}
    A& 0 \\
    0 & A^\top
\end{bmatrix}
\begin{bmatrix}
    \vec{q} \\
    \vec{p}
\end{bmatrix}
+\begin{bmatrix}
    0 \\
    \mathbf{f}(\vec{q})
\end{bmatrix}
= 0
$$

### Steady-State Example - Liquid Flow Loop

Suppose we have a system with a simple closed loop. There is a large buffer tank with a fixed volume of liquid.

From the tank, there is a recirculation pump which drives flow through a pipeline, to a high level above the tank.

On the return to the tank, there are flow control valves in parallel which increase the back pressure in the loop.



![Oil Lumped Sim-PFD](images/Oil%20Lumped%20Sim-PFD.png)

![Oil Lumped Sim-INCIDENCE GRAPH](images/Oil%20Lumped%20Sim-INCIDENCE%20GRAPH%20(SIMPLIFIED).png)

In [302]:
import numpy as np
from scipy.optimize import fsolve
from utils import pump_pressure_rise, darcy_weisbach_pressure_drop, valve_pressure_drop

In [303]:
N_NODES = 6
N_BRANCHES = 6

INCIDENCE_MATRIX = np.zeros((N_NODES, N_BRANCHES))

# pump branch
INCIDENCE_MATRIX[0, 0] = -1
INCIDENCE_MATRIX[1, 0] = +1

# trench branch
INCIDENCE_MATRIX[1, 1] = -1
INCIDENCE_MATRIX[2, 1] = 1

# dut branch
INCIDENCE_MATRIX[2, 2] = -1
INCIDENCE_MATRIX[3, 2] = 1

# fcv branch
INCIDENCE_MATRIX[3, 3] = -1
INCIDENCE_MATRIX[4, 3] = 1

# return to tank branch
INCIDENCE_MATRIX[4, 4] = -1
INCIDENCE_MATRIX[5, 4] = 1

# tank to pump branch
INCIDENCE_MATRIX[5, 5] = -1
INCIDENCE_MATRIX[0, 5] = 1

print(INCIDENCE_MATRIX)

[[-1.  0.  0.  0.  0.  1.]
 [ 1. -1.  0.  0.  0.  0.]
 [ 0.  1. -1.  0.  0.  0.]
 [ 0.  0.  1. -1.  0.  0.]
 [ 0.  0.  0.  1. -1.  0.]
 [ 0.  0.  0.  0.  1. -1.]]


In [304]:
DENSITY = 880  # kg/m^3
STATIC_HEAD_PRESSURE = DENSITY * 9.81 * 2.0
DUT_PIPE_FRICTION_FACTOR = 0.02  # stainless steel
DUT_PIPE_DIAMETER = 0.1524  # 6in, m
DUT_PIPE_LENGTH = 5.0  # m
VALVE_POSITION = 0.8  # fraction of valve open
PUMP_SPEED = 1.0  # fraction of pump speed


def calculate_residuals(design_variables):
    flowrates = design_variables[:N_BRANCHES]
    pressures = design_variables[N_BRANCHES:]
    pressures[5] = STATIC_HEAD_PRESSURE  # assume the tank pressure is approximately atmospheric  # assume the tank pressure is approximately atmospheric

    delta_pressures = np.array(
        [
            +pump_pressure_rise(flowrates[0], PUMP_SPEED),  # pump branch
            -STATIC_HEAD_PRESSURE,  # trench branch
            -darcy_weisbach_pressure_drop(
                flowrates[2],
                DUT_PIPE_FRICTION_FACTOR,
                DUT_PIPE_LENGTH,
                DUT_PIPE_DIAMETER,
            ),  # dut branch
            -valve_pressure_drop(flowrates[3], VALVE_POSITION),  # fcv branch
            +STATIC_HEAD_PRESSURE,  # return to tank branch
            0.0,  # tank to pump branch
        ]
    )

    residuals_flowrates = INCIDENCE_MATRIX @ flowrates

    residuals_pressures = INCIDENCE_MATRIX.T @ pressures - delta_pressures

    return np.hstack(
        [residuals_flowrates, residuals_pressures]
    )  # solution when objective is zero


x0 = np.random.rand((N_BRANCHES + N_NODES))
sol = fsolve(calculate_residuals, x0, full_output=True, xtol=1e-7, maxfev=int(1e8))
if sol[2] != 1:
    print(sol[3])
flowrates_sol = np.round(sol[0][:N_BRANCHES], 5)
pressures_sol = np.round(sol[0][N_BRANCHES:], 5)

print(f"flowrates (l/s)\n" f"---------------\n" f"{flowrates_sol * 1000}\n")
print(f"Pressures (barg)\n" f"----------------\n" f"{pressures_sol / 1e5}\n")

print("Pressure at the pump inlet (barg):", pressures_sol[0] / 1e5)
print("Pressure at the pump outlet (barg):", pressures_sol[1] / 1e5)

flowrates (l/s)
---------------
[175.49 175.49 175.49 175.49 175.49 175.49]

Pressures (barg)
----------------
[0.172656   1.4723202  1.2996642  1.14784296 0.         0.172656  ]

Pressure at the pump inlet (barg): 0.17265599999999998
Pressure at the pump outlet (barg): 1.4723202043999999
